In [1]:
import pandas as pd
import bnlearn as bn
import sys
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OrdinalEncoder,FunctionTransformer
from sklearn.model_selection import train_test_split
from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score, BicScore
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination,BeliefPropagation
from sklearn.metrics import accuracy_score
from pgmpy.estimators import BayesianEstimator
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing


In [2]:
silo1 = pd.read_csv(
    "/Users/joaoalmeida/Desktop/tese_local/chvng/CHVNGE_OBSCARE_WCR.csv", sep=";"
)

In [3]:
silo1.drop_duplicates(inplace=True)

In [4]:
dup_keys=["IDENTIFICADOR","DATA_PARTO","IDADE_MATERNA"]

In [5]:
def remove_duplicates_keep_higher_info(x):
    
    if len(x)==1:
        return x.index[0]
    else:
        null_value=0
        for k,r in x.iterrows():
         #   print(k)
            if r.isnull().sum()>null_value:
                null_value=r.isnull().sum()
                return_index=k
       # print(return_index)
        return return_index
       # print(x.name)
        #print(x.index)
        #print(x.isnull().sum(axis=1))
        #print(x.index[0])
        #print(x.iloc[0,:])
        #print(x.iloc[x.index[0],:])

In [6]:
silo_unique_index=silo1.groupby(dup_keys, group_keys=False).apply(lambda x: remove_duplicates_keep_higher_info(x))
#x.loc[x.B.idxmax()]

In [16]:
silo_1_unique=silo1.iloc[silo_unique_index.values,:]

In [17]:
silo_1_unique.columns.to_list()

['IDENTIFICADOR',
 'DATA_PARTO',
 'IDADE_MATERNA',
 'GS',
 'PESO_INICIAL',
 'PESO_ADMISSAO_INTERNAMENTO',
 'IMC',
 'CIGARROS',
 'ALCOOL',
 'ESTUPEFACIENTES',
 'A_PARA',
 'A_GESTA',
 'EUTOCITO_ANTERIOR',
 'VENTOSAS_ANTERIOR',
 'FORCEPS_ANTERIOR',
 'CESARIANAS_ANTERIOR',
 'CESARIANAS_MOTIVO_ANTERIOR',
 'TIPO_GRAVIDEZ',
 'VIGIADA',
 'NUMERO_CONSULTAS_PRE_NATAL',
 'VIGIADA_HOSPITAL',
 'VIGIADA_PARICULAR',
 'VIGIADA_CENTRO_SAUDE',
 'VIGIADA_NESTE_HOSPITAL',
 'ESTIMATIVA_PESO_ECO_24',
 'ESTIMATIVA_PESO_ECO_25',
 'ESTIMATIVA_PESO_ECO_26',
 'ESTIMATIVA_PESO_ECO_27',
 'ESTIMATIVA_PESO_ECO_28',
 'ESTIMATIVA_PESO_ECO_29',
 'ESTIMATIVA_PESO_ECO_30',
 'ESTIMATIVA_PESO_ECO_31',
 'ESTIMATIVA_PESO_ECO_32',
 'ESTIMATIVA_PESO_ECO_33',
 'ESTIMATIVA_PESO_ECO_34',
 'ESTIMATIVA_PESO_ECO_35',
 'ESTIMATIVA_PESO_ECO_36',
 'ESTIMATIVA_PESO_ECO_37',
 'ESTIMATIVA_PESO_ECO_38',
 'ESTIMATIVA_PESO_ECO_39',
 'ESTIMATIVA_PESO_ECO_40',
 'ESTIMATIVA_PESO_ECO_41',
 'ESTIMATIVA_PESO_ECO_42',
 'APRESENTACAO_42',
 'APRESENT

In [8]:
nulls=silo_1_unique.isnull().sum()/len(silo_1_unique)*100

In [9]:
unique_values={}
standard_map={"ADEQUADA":"A","LIMITE":"L","DESCONHECIDO,RH_DESCONHECIDO":np.nan,"DESCONHECIDO,":np.nan,"SIM":"S","NAO":"N","-1":np.nan,-1:np.nan,"NS":np.nan,",":np.nan,"Sim":"S","UNKNOWN":np.nan,"Desconhecida":np.nan,"Desconhecido":np.nan,"DESCONHECIDO":np.nan," ":np.nan,"Desconhecido,":np.nan,"  ":np.nan}
def standardize_null(x,mapping):
    if x in mapping.keys():
        return mapping[x]
    if pd.isna(x):
        return np.nan
    return x

for col in silo_1_unique.columns:
    silo_1_unique[col]=silo_1_unique[col].apply(standardize_null,mapping=standard_map)
        

In [10]:
drop_cols=["IDENTIFICADOR","DATA_PARTO","G_TERAPEUTICA","NUM_RN"]

In [11]:
for col_idx,k in enumerate(nulls):
    if k>80:
       # print(k,silo_1_unique.columns[col_idx])
        drop_cols.append(silo_1_unique.columns[col_idx])

In [12]:
silo_1_unique.drop(columns=drop_cols,inplace=True)

In [13]:
len(silo_1_unique.columns)

22

In [15]:
ord_cols=["A_PARA","A_GESTA","EUTOCITO_ANTERIOR","VENTOSAS_ANTERIOR","FORCEPS_ANTERIOR","CESARIANAS_ANTERIOR","BISHOP_SCORE",
"BISHOP_CONSISTENCIA",
"BISHOP_DESCIDA",
"BISHOP_DILATACAO",
"BISHOP_EXTINCAO",
"BISHOP_POSICAO"]

int_cols = [
    "IDADE_MATERNA",
    "PESO_INICIAL",
    "IMC",
    "NUMERO_CONSULTAS_PRE_NATAL",
    "IDADE_GESTACIONAL_ADMISSAO",
    "SEMANAS_GESTACAO_PARTO",
    "PESO_ADMISSAO_INTERNAMENTO",
    "ESTIMATIVA_PESO_ECO_30",
    "ESTIMATIVA_PESO_ECO_31",
    "ESTIMATIVA_PESO_ECO_32",
    "ESTIMATIVA_PESO_ECO_24",
"ESTIMATIVA_PESO_ECO_25",
"ESTIMATIVA_PESO_ECO_26",
"ESTIMATIVA_PESO_ECO_27",
"ESTIMATIVA_PESO_ECO_28",
"ESTIMATIVA_PESO_ECO_29",
"ESTIMATIVA_PESO_ECO_33",
"ESTIMATIVA_PESO_ECO_34",
"ESTIMATIVA_PESO_ECO_35",
"ESTIMATIVA_PESO_ECO_36",
"ESTIMATIVA_PESO_ECO_37",
"ESTIMATIVA_PESO_ECO_38",
"ESTIMATIVA_PESO_ECO_39",
"ESTIMATIVA_PESO_ECO_40",
"ESTIMATIVA_PESO_ECO_41"]
cat_cols = list(set(silo_1_unique.columns)  - set(int_cols) -set(ord_cols))

In [16]:
ord_cols=[col for col in ord_cols if col  in silo_1_unique.columns ]
int_cols=[col for col in int_cols if col  in silo_1_unique.columns ]


In [17]:
cat_cols

['VIGIADA_NESTE_HOSPITAL',
 'APRESENTACAO_NO_PARTO',
 'GS',
 'VIGIADA',
 'TIPO_GRAVIDEZ',
 'APRESENTACAO_ADMISSAO',
 'TRAB_PARTO_NO_PARTO',
 'TIPO_PARTO',
 'GRUPO_ROBSON',
 'VIGIADA_CENTRO_SAUDE',
 'APRESENTACAO_30',
 'TRAB_PARTO_ENTRADA_ESPONTANEO']

In [18]:
silo_1_unique[cat_cols]=silo_1_unique[cat_cols].astype(str)

In [21]:
numeric_transformer = Pipeline(
     steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        ),("kbins",KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile'))
    ]
)
ordinal_transformer= Pipeline(
    steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        )#,
    ]
)

categorical_transformer = Pipeline(
    steps=[ #("fun_str", fun_str),
       
        (
            "imputer",
            SimpleImputer(
                missing_values="nan", strategy="constant", fill_value="Unknown"
            )
        ),

                ("ordinalEncoder", OrdinalEncoder())

    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, int_cols),
        ("ord",ordinal_transformer,ord_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)
bn_learn_pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

In [22]:
X_treated_bn=bn_learn_pipeline.fit_transform(silo_1_unique)

In [23]:
import joblib
joblib.dump(bn_learn_pipeline, 'pipeline.sav')

['pipeline.sav']

In [24]:
def transfrom_array_to_df_onehot(pl,nparray):
    col_list=[]
    col_list_int = pl["preprocessor"].transformers_[0][2] #changes col location
    #print(col_list_int)
    ordinal_col=pl["preprocessor"].transformers[1][2]
    original_col=pl["preprocessor"].transformers[2][2]
    col_list=col_list_int+ordinal_col
  
    col_list=col_list+original_col
    df1 = pd.DataFrame(nparray, columns=col_list)
    return df1

In [28]:
df_bn=transfrom_array_to_df_onehot(bn_learn_pipeline,X_treated_bn)
df_bn=df_bn.astype(float)
#df_bn.replace("\.","_",regex=True,inplace=True)

In [19]:
numeric_transformer = Pipeline(
     steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        ),("kbins",KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile'))
    ]
)
ordinal_transformer= Pipeline(
    steps=[#("dummy",'passthrough')
         (
            "imputer",
            SimpleImputer(
                missing_values=np.nan, strategy="median"
            )
        )#,
    ]
)

categorical_transformer = Pipeline(
    steps=[ #("fun_str", fun_str),
       
        (
            "imputer",
            SimpleImputer(
                missing_values="nan", strategy="constant", fill_value="Unknown"
            )
        ),

                ("ordinalEncoder", OrdinalEncoder())

    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, int_cols),
        ("ord",ordinal_transformer,ord_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)
bn_learn_pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

In [20]:
X_treated_bn=bn_learn_pipeline.fit_transform(silo_1_unique)

In [21]:
import joblib
joblib.dump(bn_learn_pipeline, 'pipeline.sav')

['pipeline.sav']

In [22]:
def transfrom_array_to_df_onehot(pl,nparray):
    col_list=[]
    col_list_int = pl["preprocessor"].transformers_[0][2] #changes col location
    #print(col_list_int)
    ordinal_col=pl["preprocessor"].transformers[1][2]
    original_col=pl["preprocessor"].transformers[2][2]
    col_list=col_list_int+ordinal_col
  
    col_list=col_list+original_col
    df1 = pd.DataFrame(nparray, columns=col_list)
    return df1

In [23]:
df_bn=transfrom_array_to_df_onehot(bn_learn_pipeline,X_treated_bn)
df_bn=df_bn.astype(str)
df_bn.replace("\.","_",regex=True,inplace=True)

In [31]:
from sklearn.covariance import MinCovDet

In [32]:
cov = MinCovDet(random_state=0).fit(df_bn)
cov.covariance_

array([[ 6.30495276e-01,  8.26748407e-03,  1.49910150e-02,
         8.32537949e-02, -6.03744160e-02, -5.81870048e-02,
        -1.15572566e-03,  0.00000000e+00,  1.45602616e-01,
         5.42524946e-04,  2.23179619e-02,  3.49179683e-02,
         3.96431589e-03,  3.30058313e-01, -1.21286991e-01,
        -1.86290482e-02,  6.54593078e-02, -4.51519548e-03,
         0.00000000e+00,  3.28401459e-02, -9.43936204e-02,
         2.27877036e-02],
       [ 8.26748407e-03,  6.42521296e-01,  5.30282534e-01,
         2.66226354e-02,  2.56026915e-02,  3.39139810e-02,
         1.63028445e-03,  0.00000000e+00,  5.57853082e-02,
         5.49148447e-04,  5.03994987e-02,  2.96508549e-02,
         2.65316376e-04,  1.49197446e-01, -2.22003195e-02,
        -1.70048604e-02, -6.85735576e-03,  7.72014203e-04,
         0.00000000e+00, -1.97203452e-02, -1.69213139e-02,
        -4.85140589e-02],
       [ 1.49910150e-02,  5.30282534e-01,  6.49530315e-01,
         2.86641038e-02,  6.05817014e-03,  1.55464859e-02,
    

In [35]:
T=cov.mahalanobis(df_bn)

In [38]:
T.shape

(6532,)

In [40]:
T.max()

87835.17681704603

In [41]:
T.argmax()

4287

In [44]:
T.argmin()

4414

In [50]:
ff=cov.get_precision()

In [ ]:
#mcd = MinCovDet().fit(X)
# compare raw robust estimates with the true location and covariance
err_loc_mcd[i, j] = np.sum(cov.location_**2)
err_cov_mcd[i, j] = cov.error_norm(np.eye(n_features))

# compare estimators learned from the full data set with true
# parameters
err_loc_emp_full[i, j] = np.sum(X.mean(0) ** 2)
err_cov_emp_full[i, j] = (
    EmpiricalCovariance().fit(X).error_norm(np.eye(n_features))
)

In [79]:
from sklearn.neighbors import LocalOutlierFactor

# identify outliers in the training dataset
lof = LocalOutlierFactor(novelty=True)
yhat = lof.fit(df_bn)

In [81]:
yhat = lof.predict(df_bn)
# select all rows that are not outliers
mask = yhat != -1
gg=df_bn.loc[mask, :]
# summarize the shape of the updated training dataset


In [68]:
nmask = yhat != 1
outs=silo_1_unique.loc[nmask, :]
# summarize the shape of the updated training dataset


In [69]:
outs

,IDADE_MATERNA,GS,PESO_INICIAL,IMC,A_PARA,A_GESTA,EUTOCITO_ANTERIOR,TIPO_GRAVIDEZ,VIGIADA,NUMERO_CONSULTAS_PRE_NATAL,...,ESTIMATIVA_PESO_ECO_30,APRESENTACAO_30,APRESENTACAO_ADMISSAO,IDADE_GESTACIONAL_ADMISSAO,TRAB_PARTO_ENTRADA_ESPONTANEO,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON
6214,30,"A,RH_POSITIVO",65.0,23.0,NaN,1,NaN,TEC,S,6.0,...,1727.0,4.0,nan,39.0,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.0,1
3171,15,"A,RH_NEGATIVO",51.0,17.9,NaN,1,NaN,ESPONTANEA,S,11.0,...,1420.0,7.0,nan,40.0,nan,Parto eutócico cefálico,Cefálica de vértice,Indução com misoprostol,40.0,2
1212,37,"O,RH_POSITIVO",60.0,22.3,3.0,4,3.0,nan,S,8.0,...,1633.0,4.0,nan,38.0,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3
4200,35,"O,RH_POSITIVO",43.0,20.5,NaN,1,NaN,ESPONTANEA,S,9.0,...,1479.0,4.0,nan,40.0,nan,Parto auxiliado por ventosa,Cefálica de vértice,Indução com misoprostol,41.0,2
3167,25,"A,RH_POSITIVO",84.0,29.4,NaN,1,NaN,ESPONTANEA,S,6.0,...,1686.0,3.0,nan,41.0,nan,Cesariana,Cefálica de vértice,Indução com misoprostol,41.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5333,30,"O,RH_POSITIVO",50.0,19.5,3.0,6,3.0,ESPONTANEA,S,5.0,...,NaN,nan,nan,35.0,nan,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,35.0,10
261,37,"A,RH_POSITIVO",52.0,19.1,NaN,1,NaN,nan,S,NaN,...,1503.0,7.0,nan,39.0,S,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,39.0,1
5262,20,"B,RH_POSITIVO",59.0,21.7,NaN,1,NaN,ESPONTANEA,S,7.0,...,1898.0,4.0,nan,40.0,nan,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,40.0,1
3124,25,"A,RH_NEGATIVO",77.0,24.9,NaN,1,NaN,nan,S,8.0,...,1638.0,7.0,nan,39.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,39.0,1


In [70]:
from sklearn.covariance import EllipticEnvelope

# identify outliers in the training dataset
ee = EllipticEnvelope(contamination=0.01)
yhat = ee.fit_predict(df_bn)

In [71]:
mask = yhat != -1
gg=df_bn.loc[mask, :]
# summarize the shape of the updated training dataset


In [72]:
gg

,IDADE_MATERNA,PESO_INICIAL,IMC,NUMERO_CONSULTAS_PRE_NATAL,IDADE_GESTACIONAL_ADMISSAO,SEMANAS_GESTACAO_PARTO,ESTIMATIVA_PESO_ECO_30,A_PARA,A_GESTA,EUTOCITO_ANTERIOR,...,VIGIADA,GRUPO_ROBSON,TIPO_PARTO,VIGIADA_NESTE_HOSPITAL,VIGIADA_CENTRO_SAUDE,APRESENTACAO_30,APRESENTACAO_ADMISSAO,APRESENTACAO_NO_PARTO,TIPO_GRAVIDEZ,GS
0,0.0,2.0,2.0,1.0,0.0,0.0,1.0,3.0,4.0,1.0,...,1.0,1.0,0.0,0.0,1.0,20.0,0.0,0.0,0.0,7.0
1,2.0,2.0,2.0,2.0,0.0,0.0,1.0,1.0,2.0,1.0,...,1.0,5.0,0.0,0.0,0.0,20.0,0.0,0.0,6.0,6.0
2,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0,...,1.0,3.0,4.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,2.0,3.0,0.0,1.0,20.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6527,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,2.0,4.0,0.0,0.0,20.0,0.0,0.0,6.0,1.0
6528,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,0.0,4.0,1.0,1.0,20.0,0.0,0.0,6.0,7.0
6529,2.0,2.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,2.0,0.0,0.0,1.0,20.0,0.0,0.0,6.0,7.0
6530,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,2.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,1.0


In [73]:
nmask = yhat != 1
outs=silo_1_unique.loc[nmask, :]
# summarize the shape of the updated training dataset


In [74]:
outs

,IDADE_MATERNA,GS,PESO_INICIAL,IMC,A_PARA,A_GESTA,EUTOCITO_ANTERIOR,TIPO_GRAVIDEZ,VIGIADA,NUMERO_CONSULTAS_PRE_NATAL,...,ESTIMATIVA_PESO_ECO_30,APRESENTACAO_30,APRESENTACAO_ADMISSAO,IDADE_GESTACIONAL_ADMISSAO,TRAB_PARTO_ENTRADA_ESPONTANEO,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON
4373,37,"A,RH_POSITIVO",53.0,22.3,4.0,6,4.0,ESPONTANEA,S,1.0,...,NaN,nan,nan,38.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3
6181,29,"AB,RH_POSITIVO",69.0,27.0,3.0,5,3.0,ESPONTANEA,S,6.0,...,NaN,1.0,apr.cefala.3,36.0,nan,Parto eutócico cefálico,Cefálica de vértice,Indução ocitócica,36.0,10
5689,40,"O,RH_POSITIVO",87.0,31.2,4.0,6,3.0,nan,S,7.0,...,NaN,nan,nan,39.0,nan,Parto eutócico cefálico,Cefálica de vértice,Indução ocitócica,40.0,4
3154,41,"A,RH_NEGATIVO",67.0,29.8,4.0,5,3.0,nan,S,8.0,...,1549.0,1.0,nan,41.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,41.0,3
3435,45,"O,RH_POSITIVO",79.0,30.9,5.0,6,5.0,nan,S,2.0,...,NaN,nan,nan,40.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,46,"O,RH_NEGATIVO",84.0,32.8,5.0,8,5.0,ESPONTANEA,S,11.0,...,1374.0,1.0,nan,40.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.0,3
4005,30,"O,RH_POSITIVO",47.0,21.2,4.0,6,4.0,ESPONTANEA,S,9.0,...,1421.0,1.0,nan,39.0,nan,Parto eutócico cefálico,Cefálica de vértice,Indução com misoprostol,39.0,4
6339,37,"A,RH_POSITIVO",60.0,22.9,4.0,5,4.0,ESPONTANEA,S,1.0,...,NaN,nan,nan,40.0,S,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.0,3
7213,39,"B,RH_POSITIVO",69.0,27.3,3.0,6,3.0,ESPONTANEA,S,6.0,...,1443.0,1.0,nan,37.0,nan,Parto eutócico cefálico,Cefálica de vértice,Indução com misoprostol,37.0,10


In [75]:
import joblib
joblib.dump(ee, 'EllipticEnvelope.sav')

['EllipticEnvelope.sav']

In [82]:
joblib.dump(lof, 'LocalOutlierFactor.sav')

['LocalOutlierFactor.sav']